In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# NonlinearBlockGS

NonlinearBlockGS applies Block Gauss-Seidel (also known as fixed-point iteration) to the
components and subsystems in the system. This is mainly used to solve cyclic connections. You
should try this solver for systems that satisfy the following conditions:

1. System (or subsystem) contains a cycle, though subsystems may.
2. System does not contain any implicit states, though subsystems may.

NonlinearBlockGS is a block solver, so you can specify different nonlinear solvers in the subsystems and they
will be utilized to solve the subsystem nonlinear problem.

Note that you may not know if you satisfy the second condition, so choosing a solver can be a trial-and-error proposition. If
NonlinearBlockGS doesn't work, then you will need to use [NewtonSolver](../../../_srcdocs/packages/solvers.nonlinear/newton).

Here, we choose NonlinearBlockGS to solve the Sellar problem, which has two components with a
cyclic dependency, has no implicit states, and works very well with Gauss-Seidel.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src33", get_code("openmdao.test_suite.components.sellar.SellarDis1withDerivatives"), display=False)

:::{Admonition} `SellarDis1withDerivatives` class definition 
:class: dropdown

{glue:}`code_src33`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src34", get_code("openmdao.test_suite.components.sellar.SellarDis2withDerivatives"), display=False)

:::{Admonition} `SellarDis2withDerivatives` class definition 
:class: dropdown

{glue:}`code_src34`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_nbgs_sellar_derivs", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_nbgs_sellar_derivs`
:::

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

prob.model.nonlinear_solver = om.NonlinearBlockGS()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

print(prob.get_val('y1'))
print(prob.get_val('y2'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('y1'), 25.58830273, .00001)
assert_near_equal(prob.get_val('y2'), 12.05848819, .00001)

This solver runs all of the subsystems each iteration, passing data along all connections
including the cyclic ones. After each iteration, the iteration count and the residual norm are
checked to see if termination has been satisfied.

You can control the termination criteria for the solver using the following options:

# NonlinearBlockGS Options

In [ ]:
om.show_options_table("openmdao.solvers.nonlinear.nonlinear_block_gs.NonlinearBlockGS")

## NonlinearBlockGS Constructor

The call signature for the `NonlinearBlockGS` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.nonlinear.nonlinear_block_gs.NonlinearBlockGS.__init__
        :noindex:
```

## Aitken relaxation

This solver implements Aitken relaxation, as described in Algorithm 1 of this paper on aerostructual design [optimization](http://www.umich.edu/~mdolaboratory/pdf/Kenway2014a.pdf).
The relaxation is turned off by default, but it may help convergence for more tightly coupled models.

## Residual Calculation

The `Unified Derivatives Equations` are formulated so that explicit equations (via `ExplicitComponent`) are also expressed
as implicit relationships, and their residual is also calculated in "apply_nonlinear", which runs the component a second time and
saves the difference in the output vector as the residual. However, this would require an extra call to `compute`, which is
inefficient for slower components. To eliminate the inefficiency of running the model twice every iteration the NonlinearBlockGS
driver saves a copy of the output vector and uses that to calculate the residual without rerunning the model. This does require
a little more memory, so if you are solving a model where memory is more of a concern than execution time, you can set the
"use_apply_nonlinear" option to True to use the original formulation that calls "apply_nonlinear" on the subsystem.


## NonlinearBlockGS Option Examples

**maxiter**

  `maxiter` lets you specify the maximum number of Gauss-Seidel iterations to apply. In this example, we
  cut it back from the default, ten, down to two, so that it terminates a few iterations earlier and doesn't
  reach the specified absolute or relative tolerance.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

nlbgs = prob.model.nonlinear_solver = om.NonlinearBlockGS()

# basic test of number of iterations
nlbgs.options['maxiter'] = 1
prob.run_model()

print(prob.model.nonlinear_solver._iter_count)

In [ ]:
assert(prob.model.nonlinear_solver._iter_count == 1)

In [ ]:
nlbgs.options['maxiter'] = 5
prob.run_model()
print(prob.model.nonlinear_solver._iter_count)

In [ ]:
assert(prob.model.nonlinear_solver._iter_count == 5)

In [ ]:
# test of number of iterations AND solution after exit at maxiter
prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

nlbgs.options['maxiter'] = 3
prob.set_solver_print()
prob.run_model()

print(prob.get_val('y1'))
print(prob.get_val('y2'))
print(prob.model.nonlinear_solver._iter_count)

In [ ]:
assert_near_equal(prob.get_val('y1'), 25.58914915, .0001)
assert_near_equal(prob.get_val('y2'), 12.05857185, .0001)
assert(prob.model.nonlinear_solver._iter_count == 3)

**atol**

  Here, we set the absolute tolerance to a looser value that will trigger an earlier termination. After
  each iteration, the norm of the residuals is calculated one of two ways. If the "use_apply_nonlinear" option
  is set to False (its default), then the norm is calculated by subtracting a cached previous value of the
  outputs from the current value.  If "use_apply_nonlinear" is True, then the norm is calculated by calling
  apply_nonlinear on all of the subsystems. In this case, `ExplicitComponents` are executed a second time.
  If this norm value is lower than the absolute tolerance `atol`, the iteration will terminate.

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

nlbgs = prob.model.nonlinear_solver = om.NonlinearBlockGS()
nlbgs.options['atol'] = 1e-4

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

print(prob.get_val('y1'))
print(prob.get_val('y2'))

In [ ]:
assert_near_equal(prob.get_val('y1'), 25.5882856302, .00001)
assert_near_equal(prob.get_val('y2'), 12.05848819, .00001)

**rtol**

  Here, we set the relative tolerance to a looser value that will trigger an earlier termination. After
  each iteration, the norm of the residuals is calculated one of two ways. If the "use_apply_nonlinear" option
  is set to False (its default), then the norm is calculated by subtracting a cached previous value of the
  outputs from the current value.  If "use_apply_nonlinear" is True, then the norm is calculated by calling
  apply_nonlinear on all of the subsystems. In this case, `ExplicitComponents` are executed a second time.
  If the ratio of the currently calculated norm to the initial residual norm is lower than the relative tolerance
  `rtol`, the iteration will terminate.

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

nlbgs = prob.model.nonlinear_solver = om.NonlinearBlockGS()
nlbgs.options['rtol'] = 1e-3

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

print(prob.get_val('y1'), 25.5883027, .00001)
print(prob.get_val('y2'), 12.05848819, .00001)

In [ ]:
assert_near_equal(prob.get_val('y1'), 25.5883027, .00001)
assert_near_equal(prob.get_val('y2'), 12.05848819, .00001)